In [1]:
import bayesnewton
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import pandas as pd
from scipy.spatial.distance import cdist
from bayes_opt import BayesianOptimization
from sklearn.neighbors import NearestNeighbors


import sys, os
sys.path.append('../../Utils')
import model_utils as mutils
%load_ext autoreload
%autoreload 2

/Users/SeanNassimiha/miniforge3/envs/mscjax_dev/lib/python3.8/site-packages/jax/_src/lib/__init__.py:33: UserWarning: JAX on Mac ARM machines is experimental and minimally tested. Please see https://github.com/google/jax/issues/5501 in the event of problems.
  warnings.warn("JAX on Mac ARM machines is experimental and minimally tested. "


In [2]:
data =  pd.read_csv('../../../Data/pv_power_df_5day_capacity_scaled.csv', index_col='datetime').drop(columns=['2657', '2828']) #DROPPING FAULTY SYSTEMS
uk_pv = pd.read_csv('../../../Data/system_metadata_location_rounded.csv')
uk_pv['ss_id_string'] = uk_pv['ss_id'].astype('str')

lats = dict(uk_pv.set_index('ss_id')['latitude_noisy'])
longs = dict(uk_pv.set_index('ss_id')['longitude_noisy'])

In [5]:
#DATA VARIABLES
# SYSTEMS_NUM = 400
# TIMESTEPS_NUM = int(len(data) / 5)
# TRAIN_FRAC = 0.9
# TEST_STATIONS = 400


# #DATA VARIABLES
SYSTEMS_NUM = 450
TIMESTEPS_NUM = int(len(data) / 500)
TRAIN_FRAC = 0.9
TEST_STATIONS = 400

In [6]:
data_multiple = data.iloc[:, :SYSTEMS_NUM][:TIMESTEPS_NUM].reset_index()

capacities = uk_pv[uk_pv.ss_id_string.isin(data_multiple.columns)].set_index('ss_id_string')['kwp'].values * 1000
stacked = mutils.stack_dataframe(data_multiple, lats, longs)

X = np.array(stacked[['epoch', 'longitude', 'latitude']])
Y = np.array(stacked[['PV']])
#Create a space-time grid from X and Y
t, R, Y = bayesnewton.utils.create_spatiotemporal_grid(X, Y)

#train test split for 3 dimensional data
t_train, t_test, R_train, R_test, Y_train, Y_test = mutils.train_split_3d(t, R, Y, train_frac = TRAIN_FRAC, split_by_day = False)

#Scale the data
scaled_values = mutils.scale_2d_train_test_data(R, Y, R_train, R_test, Y_train, Y_test )
R_scaler, R_scaled, R_train_scaled, R_test_scaled, _, _, _, _ = scaled_values


In [7]:
data_unseen = data.iloc[:, SYSTEMS_NUM:SYSTEMS_NUM+TEST_STATIONS][:TIMESTEPS_NUM].reset_index()

capacities_unseen = uk_pv[uk_pv.ss_id_string.isin(data_unseen.columns)].set_index('ss_id_string')['kwp'].values * 1000
stacked_unseen = mutils.stack_dataframe(data_unseen, lats, longs)

del data,uk_pv

X_unseen = np.array(stacked_unseen[['epoch', 'longitude', 'latitude']])
Y_unseen = np.array(stacked_unseen[['PV']])

t, R_unseen, Y_unseen = bayesnewton.utils.create_spatiotemporal_grid(X_unseen, Y_unseen)

R_scaled_unseen = np.tile(R_scaler.transform(R_unseen[0]), (R_unseen.shape[0],1, 1)) #renormalise R and project across time



In [8]:
def evaluate_IDW(power, time_factor, neighbours_space, neighbours_time, t_train, R_train, Y_train, t_test, R_test, Y_test, capacities_test):
    '''
    Function that calculates the IWD prediction for a single helf out system
    power - power of distance metric
    time_factor - factor that governs the weighted sum between time and space dimension
    
    returns:
    - MAE of the predictor (averages over all systems)
    - prediction for that specific location and timestep
    
    '''
    print('beginning IDW')
    
    print('get space neighbours')
    nbrs_space = NearestNeighbors(n_neighbors=neighbours_space, algorithm='ball_tree').fit(R_train)
    distances_neighbours_space, idx_neighbours_space = nbrs_space.kneighbors(R_test)
    
    print('get time neighbours')
    nbrs_time = NearestNeighbors(n_neighbors=neighbours_time, algorithm = 'kd_tree', p=1).fit(t_train)
    distances_neighbours_time, idx_neighbours_time = nbrs_time.kneighbors(t_test)

    distances = np.add.outer(distances_neighbours_space.T, distances_neighbours_time.T).swapaxes(0,3)
    del distances_neighbours_space, distances_neighbours_time
    weights = (1 / distances)** power
    del distances
    norm_weights = weights / np.sum(weights, axis = (2,3))[:,:, np.newaxis, np.newaxis]
    del weights
    y_neighbours = Y_train[idx_neighbours_time][:,:, idx_neighbours_space].swapaxes(1,2)
    del idx_neighbours_space, idx_neighbours_time
    predictions = np.sum(norm_weights * y_neighbours, axis = (2,3)) 
    del norm_weights
    Y_MW = Y_test * capacities_test
    predicted_MW = predictions * capacities_test
    MAE = np.sqrt(np.nanmean((Y_MW - predicted_MW)**2))
    print('Terminated IDW')
    return MAE, predictions

In [11]:
nbrs_time = NearestNeighbors(n_neighbors=10, algorithm = 'kd_tree', p=1).fit(t_train)
distances_neighbours_time, idx_neighbours_time = nbrs_time.kneighbors(t)

In [31]:
print(distances_neighbours_time.shape)
distances_neighbours_time[10]

(141, 10)


array([0., 1., 1., 2., 3., 3., 4., 4., 5., 5.])

In [30]:
print(idx_neighbours_time.shape)
idx_neighbours_time[10]

(141, 10)


array([10, 11,  9,  8, 12,  7, 13,  6,  5, 14])

In [20]:
idx_neighbours_time[-1]

array([125, 124, 123, 122, 121, 120, 119, 118, 117, 116])

In [25]:
t.shape

(141, 1)

In [26]:
t_train.shape

(126, 1)

In [32]:
t_train[0]

DeviceArray([0.], dtype=float64)

In [39]:
t[(t>t_train[0] - 20 ) & (t<t_train[0] + 20) ]

DeviceArray([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.,
             12., 13., 14., 15., 16., 17., 18., 19.], dtype=float64)

In [ ]:
# power = 2
# time_factor = 1/10
# neighbours_space = 10
# neighbours_time = 10
# MAE, predictions = evaluate_IDW(power, time_factor, neighbours_space, neighbours_time, t_train, R_train_scaled[0], Y_train[:,:,0], t, R_scaled_unseen[0], Y_unseen[:,:,0], capacities_unseen)

# plt.plot(predictions[:,10])
# plt.plot(Y_unseen[:,10])




In [ ]:
def opt_IDW(power, time_factor, n_space, n_time):
    '''
    Wrapper function for bayesian opt of evaluate IDW
    '''
    
    neighbours_space = int(n_space)
    neighbours_time = int(n_time)
    
    MAE, predictions = evaluate_IDW(power, time_factor, neighbours_space, neighbours_time, t_train, R_train_scaled[0], Y_train[:,:,0], t, R_scaled_unseen[0], Y_unseen[:,:,0], capacities_unseen)
    #return the negative value 
    return - MAE



In [ ]:
optimiser = BayesianOptimization(
        f = opt_IDW,
        pbounds = {'power' : (0.01, 5), 'time_factor': (0.01, 1), 'n_space': (2, 20), 'n_time': (5, 100)},
        verbose = 2
        )
optimiser.maximize(n_iter = 50, init_points = 6)
print('final result',optimiser.max)